In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import json
nltk.download('punkt')
nltk.download('stopwords')
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import AdamW

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pickle
#load the model
with open('/kaggle/input/bert_tuned/transformers/bert_tune/1/model_BERT.pkl','rb') as f:
  model = pickle.load(f)


In [ ]:
# Step 2: Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=7)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
def load_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


Train_data = load_json_file('/kaggle/input/emotion/train_file.json')
Val_data = load_json_file('/kaggle/input/emotion/val_file.json')
# x=600
Train_data = Train_data
Val_data = Val_data

l=(Train_data[0])

In [ ]:
(l)

{'episode': 'utterance_3492',
 'speakers': ['Phoebe', 'Eric', 'Phoebe', 'Eric', 'Phoebe'],
 'emotions': ['surprise', 'fear', 'surprise', 'sadness', 'disgust'],
 'utterances': ['You-you\x85you had sex with Ursula?!',
  'Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and',
  "You didn't notice she was wearing different clothes?!",
  'Well I was just so excited to see you.',
  "Oh. Ew! Ew! Ew! Ugh! Y'know what? This is too weird."],
 'triggers': [1.0, 1.0, 0.0, 0.0, 0.0]}

In [ ]:
train_X=[]
train_Y=[]
for i in Train_data:
    train_X.extend([j for j in i['utterances']])
    train_Y.extend([j for j in i['triggers']])

val_X=[]
val_Y=[]
for i in Val_data:
    val_X.extend([j for j in i['utterances']])
    val_Y.extend([j for j in i['triggers']])




In [ ]:
# # ohe = { 'anger': [1, 0, 0, 0, 0, 0, 0], 'disgust': [0, 1, 0, 0, 0, 0, 0],'fear': [0, 0, 1, 0, 0, 0, 0],'joy': [0, 0, 0, 1, 0, 0, 0],'neutral': [0, 0, 0, 0, 1, 0, 0], 'sadness': [0, 0, 0, 0, 0, 1, 0],'surprise': [0, 0, 0, 0, 0, 0, 1]}
# ohe = {0:[0],1:[1]}
# for i in range(len(train_Y)):
#     k=train_Y[i]
#     train_Y[i]=ohe[k]

# for i in range(len(val_Y)):
#     k=val_Y[i]
#     val_Y[i]=ohe[k]

# print(train_Y)
l=[]
for i in train_Y:
    if(i==None):
        print(i)
        l.append(0.0)
        continue

    l.append(i)

l1=[]
for i in val_Y:
    if(i==None):
        print(i)
        l1.append(0.0)
        continue

    l1.append(i)


train_Y=l
val_Y=l1
# print(train_Y)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [ ]:


def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

train_X = [preprocess_text(sentence) for sentence in train_X]
val_X = [preprocess_text(sentence) for sentence in val_X]


In [ ]:
train_X[0]
val_X[0]

'hey hold minute hold second think pearls nice'

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
import torch
from sklearn.model_selection import train_test_split

# Define device (CPU or GPU)
# device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
learning_rate = 2e-5
epochs = 4
# import gc


# Load BERT tokenizer (choose a pre-trained model for classification)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Function to preprocess data (tokenize and convert to tensors)
def preprocess_data(sentences, labels):
  tokenized_data = tokenizer(sentences, padding="max_length", truncation=True)
  input_ids = torch.tensor(tokenized_data['input_ids'], dtype=torch.long)
  attention_mask = torch.tensor(tokenized_data['attention_mask'], dtype=torch.long)
  labels = torch.tensor(labels, dtype=torch.long)
  return TensorDataset(input_ids, attention_mask, labels)

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_X, val_X, train_Y, val_Y

# Preprocess train and validation data
train_data = preprocess_data(X_train, y_train)
val_data = preprocess_data(X_val, y_val)

# Create DataLoaders
b=16
train_loader = DataLoader(train_data, batch_size=b, shuffle=True)
val_loader = DataLoader(val_data, batch_size=b)

# Load pre-trained BERT model for sequence classification
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(train_Y[0]))  # Adjust num_labels for your emotion set
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss().float()

import torch

train_batch_losses, val_batch_losses = [], []
train_loss, val_loss = [], []

for epoch in range(epochs):
    torch.cuda.empty_cache()
    print(epoch)
    # Training phase
    model.train()
    epoch_train_loss = 0
    i=0
    for batch_idx, batch in enumerate(train_loader):
        print(f"Batch {i+1}/{len(train_loader)}")
        i+=1
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()  # Reset gradients to zero before starting backpropagation
        outputs = model(input_ids, attention_mask=attention_mask)
        labels = labels
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        batch_loss = loss.item()
        epoch_train_loss += batch_loss
        train_batch_losses.append(batch_loss)  # Store batch loss
        torch.cuda.empty_cache()


    avg_train_loss = epoch_train_loss / len(train_loader)
    train_loss.append(avg_train_loss)  # Store average epoch training loss

    # Validation phase
    model.eval()
    epoch_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            labels=labels
            val_batch_loss = criterion(outputs.logits, labels).item()
            epoch_val_loss += val_batch_loss
            val_batch_losses.append(val_batch_loss)  # Store batch loss
            torch.cuda.empty_cache()


    avg_val_loss = epoch_val_loss / len(val_loader)
    val_loss.append(avg_val_loss)  # Store average epoch validation loss

    # Print epoch information with average training and validation losses
    print(f"Epoch {epoch+1}/{epochs}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

print("Training complete!")

# # Import libraries for plotting (e.g., matplotlib)
# import matplotlib.pyplot as plt

# # Plot epoch vs train and validation loss
# plt.plot(range(epochs), train_loss, label='Train Loss')
# plt.plot(range(epochs), val_loss, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('BERT Model - Train vs. Validation Loss')
# plt.legend()
# plt.show()



0
Batch 1/3685
Batch 2/3685
Batch 3/3685
Batch 4/3685
Batch 5/3685
Batch 6/3685
Batch 7/3685
Batch 8/3685
Batch 9/3685
Batch 10/3685
Batch 11/3685
Batch 12/3685
Batch 13/3685
Batch 14/3685
Batch 15/3685
Batch 16/3685
Batch 17/3685
Batch 18/3685
Batch 19/3685
Batch 20/3685
Batch 21/3685
Batch 22/3685
Batch 23/3685
Batch 24/3685
Batch 25/3685
Batch 26/3685
Batch 27/3685
Batch 28/3685
Batch 29/3685
Batch 30/3685
Batch 31/3685
Batch 32/3685
Batch 33/3685
Batch 34/3685
Batch 35/3685
Batch 36/3685
Batch 37/3685
Batch 38/3685
Batch 39/3685
Batch 40/3685
Batch 41/3685
Batch 42/3685
Batch 43/3685
Batch 44/3685
Batch 45/3685
Batch 46/3685
Batch 47/3685
Batch 48/3685
Batch 49/3685
Batch 50/3685
Batch 51/3685
Batch 52/3685
Batch 53/3685
Batch 54/3685
Batch 55/3685
Batch 56/3685
Batch 57/3685
Batch 58/3685
Batch 59/3685
Batch 60/3685
Batch 61/3685
Batch 62/3685
Batch 63/3685
Batch 64/3685
Batch 65/3685
Batch 66/3685
Batch 67/3685
Batch 68/3685
Batch 69/3685
Batch 70/3685
Batch 71/3685
Batch 72/3685

KeyboardInterrupt: 

In [ ]:
# graph batch losses

import matplotlib.pyplot as plt

# plt.plot(range(len(train_batch_losses)), train_batch_losses, label='Train Batch Loss')
# plt.plot(range(len(val_batch_losses)), val_batch_losses, label='Validation Batch Loss')
# smoothing the curve
n=100
train_batch_losses_smoothed = [sum(train_batch_losses[i:i+n])/n for i in range(len(train_batch_losses)-n)]
plt.plot(range(len(train_batch_losses_smoothed)), train_batch_losses_smoothed, label='Train Batch Loss Smoothed')



plt.xlabel('Batch')
plt.ylabel('Loss')
plt.title('BERT Model - Train vs. Validation Batch Loss')
plt.legend()
plt.show()


print(val_batch_losses)

#Do the same for validation batch losses
n=300
val_batch_losses_smoothed = [sum(val_batch_losses[i:i+n])/n for i in range(len(val_batch_losses)-n)]
print(val_batch_losses_smoothed)
plt.plot(range(len(val_batch_losses_smoothed)), val_batch_losses_smoothed, label='Validation Batch Loss Smoothed')
plt.xlabel('Batch')
plt.ylabel('Loss')
plt.title('BERT Model - Train vs. Validation Batch Loss')
plt.legend()
plt.show()




In [ ]:
# # save the model file
# model.save_pretrained('model')
# tokenizer.save_pretrained('model')

#  save the model file using pickle
import pickle
with open('M2_model_BERT.pkl', 'wb') as f:
    pickle.dump(model, f)


# #load the model file using pickle
# with open('model_BERT.pkl', 'rb') as f:
#     model = pickle.load(f)

In [ ]:
#give code to evaluate the model
# Evaluate the model
from sklearn.preprocessing import OneHotEncoder
import torch
import numpy as np
from tensorflow.keras.utils import to_categorical
# from torch.nn.functional import to_categorical

model.eval()
predictions = []
true_labels = []

i=0
with torch.no_grad():
    for batch in val_loader:
        print(f"Batch {i} of {len(val_loader)}")
        i+=1
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions.extend(torch.argmax(outputs.logits, axis=1).tolist())
        true_labels.extend(labels.tolist())


# Calculate accuracy


# correct = sum([1 if p == t else 0 for p, t in zip(predictions, true_labels)])
# print(correct)
# accuracy = correct / len(true_labels)
# print(f"Accuracy: {accuracy:.4f}")


2024-04-20 16:55:46.677472: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 16:55:46.677601: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 16:55:46.828648: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Batch 0 of 456
Batch 1 of 456
Batch 2 of 456
Batch 3 of 456
Batch 4 of 456
Batch 5 of 456
Batch 6 of 456
Batch 7 of 456
Batch 8 of 456
Batch 9 of 456
Batch 10 of 456
Batch 11 of 456
Batch 12 of 456
Batch 13 of 456
Batch 14 of 456
Batch 15 of 456
Batch 16 of 456
Batch 17 of 456
Batch 18 of 456
Batch 19 of 456
Batch 20 of 456
Batch 21 of 456
Batch 22 of 456
Batch 23 of 456
Batch 24 of 456
Batch 25 of 456
Batch 26 of 456
Batch 27 of 456
Batch 28 of 456
Batch 29 of 456
Batch 30 of 456
Batch 31 of 456
Batch 32 of 456
Batch 33 of 456
Batch 34 of 456
Batch 35 of 456
Batch 36 of 456
Batch 37 of 456
Batch 38 of 456
Batch 39 of 456
Batch 40 of 456
Batch 41 of 456
Batch 42 of 456
Batch 43 of 456
Batch 44 of 456
Batch 45 of 456
Batch 46 of 456
Batch 47 of 456
Batch 48 of 456
Batch 49 of 456
Batch 50 of 456
Batch 51 of 456
Batch 52 of 456
Batch 53 of 456
Batch 54 of 456
Batch 55 of 456
Batch 56 of 456
Batch 57 of 456
Batch 58 of 456
Batch 59 of 456
Batch 60 of 456
Batch 61 of 456
Batch 62 of 456
Ba

In [ ]:
print(predictions)
print(true_labels)

In [ ]:
correct = sum([1 if p == t else 0 for p, t in zip(predictions, true_labels)])
accuracy = correct / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.8482


In [ ]:
#give f1 score
from sklearn.metrics import f1_score
f1 = f1_score(true_labels, predictions, average='weighted')
print(f"F1 Score: {f1:.4f}")


F1 Score: 0.7785
